In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras import models
from keras import layers
from keras import regularizers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
full_data = np.load('model_data.npy')
movies = np.array(pd.read_csv('movie_data.csv',
                              usecols=['movieId']))
movies_full = np.array(pd.read_csv('movie_data.csv'))

In [3]:
full_data

array([[1264, 3918,  221, 1293],
       [1232, 2866, 1890, 1888],
       [ 326, 1734, 2723, 1751],
       ...,
       [ 371, 1921, 4035, 1343],
       [8125,  438, 1458, 5854],
       [6425, 7051, 1755, 2902]])

In [4]:
X = full_data[:,:3]
y = full_data[:,3]
y = y.reshape(-1,1)

In [5]:
movies

array([[    1],
       [    2],
       [    3],
       ...,
       [27276],
       [27277],
       [27278]])

In [6]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(movies)

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [7]:
y = encoder.transform(y).toarray()

In [8]:
X.shape

(138493, 3)

In [9]:
y.shape

(138493, 27278)

In [10]:
#X_train, X_test, y_train, y_test = train_test_split(
#    X, y, test_size=0.25, random_state=0)

In [11]:
MAX_MOVIEID = movies.shape[0]
NUM_MOVIES = movies.shape[0]
NUM_ENCODINGS = 32

In [12]:
#regularizers.l2(.001)
network = models.Sequential()
network.add(layers.Embedding(MAX_MOVIEID, NUM_ENCODINGS))
#network.add(layers.Flatten())
#network.add(layers.Dense(units=64, activation='relu'))
network.add(layers.Bidirectional(layers.LSTM(NUM_ENCODINGS, activation="tanh", dropout=0.2, recurrent_dropout=0.2)))
#network.add(layers.LSTM(NUM_ENCODINGS, activation="tanh", dropout=0.2, recurrent_dropout=0.2))
#network.add(layers.Flatten())
network.add(layers.Dense(NUM_MOVIES, activation='softmax'))
network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
#regularizers.l2(.001)
#network = models.Sequential()
#network.add(layers.Embedding(MAX_MOVIEID, NUM_ENCODINGS, embeddings_regularizer=None, batch_input_shape=(128, 3)))
#network.add(layers.Flatten())
#network.add(layers.Dense(units=64, activation='relu'))
#network.add(layers.LSTM(units=NUM_ENCODINGS, recurrent_regularizer=None, return_sequences=False))
#network.add(layers.LSTM(units=NUM_ENCODINGS, recurrent_regularizer=None, return_sequences=True))
#network.add(layers.Flatten())
#network.add(layers.Dense(units=MAX_MOVIEID, activation='softmax'))
#network.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [13]:
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          872896    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
dense_1 (Dense)              (None, 27278)             1773070   
Total params: 2,662,606
Trainable params: 2,662,606
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = network.fit(
    X, y,
    epochs=3,
    shuffle=True,
    validation_split=0
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
138493/138493 [==============================] - 268s 2ms/step - loss: 7.6304 - acc: 0.0062
Epoch 2/3
138493/138493 [==============================] - 383s 3ms/step - loss: 7.2303 - acc: 0.0070
Epoch 3/3
138493/138493 [==============================] - 392s 3ms/step - loss: 7.1068 - acc: 0.0077


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [15]:
pred = network.predict(np.array([[1,0,0]]))

In [16]:
pred = pred[0]

In [17]:
to_decode = [1 if p == max(pred) else 0 for p in pred]

In [18]:
d = np.array([to_decode])

In [19]:
encoder.inverse_transform(d)

array([[258]])

In [21]:
movies_full[257,]

array([258, 'Star Wars: Episode IV - A New Hope', 'tt0076759'],
      dtype=object)

In [38]:
indices = pred.argsort()[-25:][::-1]+1

In [40]:
for index in indices:
    print(index)

258
1185
844
1
603
316
2773
1174
768
1172
353
32
524
2487
294
588
1243
1196
1662
766
109
2631
477
1945
641


In [ ]:
max(pred)

In [ ]:
np.sort(pred)[-1]

In [ ]:
to_decode2 = [1 if p == np.sort(pred)[-2] else 0 for p in pred]

In [ ]:
encoder.inverse_transform(np.array([to_decode2]))

In [ ]:
movies_full[340,]

In [41]:
network.save('v_0.h5')

In [43]:
from keras.models import load_model

In [44]:
test_load = load_model('./v_0.h5')

In [45]:
test_load.predict(np.array([[1,0,0]]))

array([[3.0208102e-03, 6.1788800e-04, 7.6529285e-04, ..., 3.0622243e-08,
        3.0566806e-08, 3.0463845e-08]], dtype=float32)